In [21]:
import pandas as pd
import numpy as np

import warnings

In [4]:
#################################### DADOS ###################################

filepath = r"C:\Users\user\Projetos\densities4risk\data\processed\database.parquet"
df = pd.read_parquet(filepath)
TICKER = 'BVSP'
df_bvsp = df[df["ticker"]==TICKER].loc[:,["datetime", "open", "high", "low", "close"]].dropna().drop_duplicates()
df_bvsp.set_index("datetime", inplace=True)

start_time = '10:00'
end_time = '17:00'
freq = '5T'  # 5 minutes

df_bvsp = df[df["ticker"]==TICKER].loc[:,["datetime", "open", "high", "low", "close"]].dropna().drop_duplicates()
df_bvsp.set_index("datetime", inplace=True)

# Create a complete time index for each day
def complete_day(group):
    day = group.index[0].date()
    full_index = pd.date_range(
        f"{day} {start_time}",
        f"{day} {end_time}",
        freq=freq
    )
    return group.reindex(full_index)

# Apply to each day
df_complete = df_bvsp.groupby(df_bvsp.index.date, group_keys=False).apply(complete_day)
# Reset index and rename column
df_complete = df_complete.reset_index().rename(columns={'index': 'datetime'})
df_complete.interpolate(method="linear", inplace=True)

df_complete["R_t"] = df_complete["close"].pct_change()
df_complete["r_t"] = np.log(df_complete["close"]) - np.log(df_complete["close"].shift(1))

C:\Users\user\AppData\Local\Temp\ipykernel_3460\127940432.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(


In [18]:
# exemple
l_inf = 0
l_sup = 2
n = 512
prob = 0.5

dSup = np.linspace(l_inf, l_sup, n) # support (grid) for Density domain
dens = np.full(n, prob) - 0.1 # density values on dSup - must be strictly positive (otherwise will truncate) and integrate to 1

In [ ]:
# Function to convert densities to log quantile density functions

if np.any(dens)<0:
    raise Exception("There are negative density values")
if np.abs(np.trapezoid(dens, dSup) - 1) > 1e-5:
    warnings.warn("Density does not integrate to 1 with tolerance of 1e-5 - renormalizing now.")
    dens = dens/np.trapezoid(dens, dSup)
if np.any(dens==0):  # FUNÇÃO NÃO CONSIDERA POSSIBILIDADE DE ZERO ENTRE LIMITES DE (SUPORTE>O)
    warnings.warn("There are some zero density values - truncating support grid so all are positive.")
    lower_bound = np.min(np.where(dens>0))
    upper_bound = np.max(np.where(dens>0))
    dSup = dSup[lower_bound : upper_bound]
    dens = dens = dens/np.trapezoid(dens, dSup) # ENTENDER POR QUE ISSO FUNCIONA


C:\Users\user\AppData\Local\Temp\ipykernel_3460\433475632.py:6: UserWarning: Density does not integrate to 1 with tolerance of 1e-5 - renormalizing now.
  warnings.warn("Density does not integrate to 1 with tolerance of 1e-5 - renormalizing now.")
